# 8.4. Interactividad: ipywidgets.

- ipywidgets nos permite hacer de forma muy sencilla pequeñas aplicaciones, con botones, dentro de los notebooks.
- Para ello usaremos el decorador interact:  https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html
- Puedes ver todos los widgets disponibles en: https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interact
import ipywidgets as widgets

¿Cómo funciona?

Básicamente, tenemos una función que recibe un parámetro

Y ese parámetro queremos que venga de una barra deslizante (por ejemplo)

En vez de tener que poner un montón de código, ipywidgets tiene un decorador que importamos @interact, para esta finalidad

In [2]:
@interact(x=1.0)
def func_to_inter(x):
    print(x)

interactive(children=(FloatSlider(value=1.0, description='x', max=3.0, min=-1.0), Output()), _dom_classes=('wi…

Cada vez que me deslizo en la barra (cambio su valor), la función se ejecuta de nuevo, arrojando un nuevo output

@interact tiene multitud de opciones (ver documentación)

Por ejemplo, si la función recibe un booleano, podemos hacer un checkbox para indicar si está a True, o False.

In [3]:
@interact(x=True, y=1.0)
def func_to_inter(x, y):
    print(x, y)

interactive(children=(Checkbox(value=True, description='x'), FloatSlider(value=1.0, description='y', max=3.0, …

Podemos definir mejor la funcionalidad del widget, instanciandolo antes:

In [7]:
@interact(
    x=widgets.IntSlider(min=-10, max=30, step=1, value=10),
    y=widgets.Dropdown(options=[('One', 1), ('Two', 2), ('Three', 3)], value=2,description='Y:')
)
def func_to_inter(x, y):
    print(x, y)

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Dropdown(description='Y:', index=…

In [8]:
@interact(
    a=widgets.FloatSlider(min=1, max=10, step=1, value=1), # amplitud
    f=widgets.FloatSlider(min=1, max=50, step=1, value=1), # frecuencia
)
def func_to_inter(a, f):
    t = np.linspace(0, 1, 1000)
    y = a*np.sin(2*np.pi*f*t)  
    plt.plot(t, y)
    plt.ylim([-10, 10])

interactive(children=(FloatSlider(value=1.0, description='a', max=10.0, min=1.0, step=1.0), FloatSlider(value=…